In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from modules.my_spark_regression import *
from modules.my_pyspark import *
from modules.my_drawer import MyDrawer

In [3]:
spark = MyPySpark(session=True, sql=True)
drawer = MyDrawer()